In [ ]:
import tweepy
import nltk
import pandas as pd
import pickle
import numpy as np 
import re 
import os
import random
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
from statistics import mode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [ ]:
# Open dataset 
df_neg = pd.read_csv('../dataset/n_out.csv')
df_pos = pd.read_csv('../dataset/p_out.csv')
# class = 0 = negative, 1 = positive\
dt_transformed = pd.concat([df_pos, df_neg])
dt_transformed


In [ ]:
all_words = [] 
documents = [] 

def processing_text_to_pos_tag(row_table):
    all_stopwords = stopwords.words('portuguese')
    t_data = re.sub("@[0-9A-za-z:]+", "", row_table[0])  # removing @user
    t_data = re.sub("https?://[A-Za-z0-9./&#]+", "", t_data)  # removing http links
    t_data = re.sub("(:..)", "", t_data)  # removing faces
    t_data = t_data.lower()
    t_data = t_data.split()
    ps = PorterStemmer()
    t_data = [ps.stem(word) for word in t_data if not word in set(all_stopwords) if len(word) > 2]
    t_data = ' '.join(t_data)
    
    return row_table[1], t_data

In [ ]:
# allowed_word_types = ["J"]

for row in dt_transformed.values:
    c,t = processing_text_to_pos_tag(row)
    documents.append( (t,c) )
    words = word_tokenize(t)
#   pos_t = nltk.pos_tag(words)
    for w in words:
        all_words.append(w)
        
save_documents = open("../pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())

save_word_features = open("../pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [ ]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [ ]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)

In [ ]:
testing_set = featuresets[4000:]
training_set = featuresets[:4000]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
# classifier.show_most_informative_features(15)

save_classifier = open("../pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)

save_classifier = open("../pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

In [ ]:
documents_f = open("../pickled_algos/documents.pickle", "rb")
documents = pickle.load(documents_f)
documents_f.close()


word_features5k_f = open("../pickled_algos/word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()




In [ ]:
open_file = open("../pickled_algos/originalnaivebayes5k.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/MNB_classifier5k.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()



open_file = open("../pickled_algos/BernoulliNB_classifier5k.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/LogisticRegression_classifier5k.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/LinearSVC_classifier5k.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/SGDC_classifier5k.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()

In [ ]:
voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier,
                                  SGDC_classifier)

In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json


def out(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)


ckey=""
csecret=""
atoken=""
asecret=""


class listener(StreamListener):

    def on_data(self, data):
        
        all_data = json.loads(data)
        self.tweet = all_data["text"]
        self.id = all_data["id"]
        self.sentiment_value, self.confidence = out(self.tweet)
        
        print(self.tweet, self.sentiment_value, self.confidence)

        if self.confidence*100 >= 80:
            output = open("twitter-out.txt","a")
            output.write(str(self.sentiment_value))
            output.write('\n')
            output.close()
        else:
            output = open("twitter-out.txt","a")
            output.write(str(2))
            output.write('\n')
            output.close()
            
        return True

    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=[""])